In [1]:
import pandas as np
import numpy as np1 
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import svm, datasets, metrics

In [2]:
def normalize(data, cat) :
    data[cat]= ((data[cat]-data[cat].min())/ (data[cat].max()-data[cat].min()))

In [5]:
df = np.read_csv('education.csv')
df_1= np.read_csv('educnorma.csv',index_col=0)

In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033 entries, 0 to 1032
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           1033 non-null   float64
 1   nb_subjets                    1033 non-null   float64
 2   Family_size                   1033 non-null   float64
 3   Internet                      1033 non-null   float64
 4   Study                         1033 non-null   float64
 5   Sleep                         1033 non-null   float64
 6   Media                         1033 non-null   float64
 7   Note_avant                    1033 non-null   float64
 8   Online_mode                   1033 non-null   float64
 9   doubts_online                 1033 non-null   float64
 10  performance_online            1033 non-null   float64
 11  Gender_Male                   1033 non-null   int64  
 12  Home_location_Urban           1033 non-null   int64  
 13  Edu

In [21]:
corr = df_1.corr()
np.io.formats.style.Styler(corr,precision=2).background_gradient(cmap='coolwarm',axis=None)

,Age,nb_subjets,Family_size,Internet,Study,Sleep,Media,Note_avant,Online_mode,doubts_online,performance_online,Gender_Male,Home_location_Urban,Education_Post Graduate,Education_School,Education_Under Graduate,Device_used_Desktop,Device_used_Laptop,Device_used_Mobile,Economic_status_Middle Class,Economic_status_Poor,Economic_status_Rich,Sport_Yes,Monitoring_Yes,Gaming_Yes,separate_room_Yes,group_study_yes,interest_Both,interest_Practical,interest_Theory,Satisfaction_online_Average,Satisfaction_online_Bad,Satisfaction_online_Good
Age,1.00,-0.01,0.02,0.07,0.01,-0.07,0.01,-0.21,0.17,0.13,0.11,-0.04,0.00,0.42,-0.15,-0.24,0.02,0.02,-0.03,0.03,-0.05,0.02,0.02,-0.04,-0.01,0.02,0.11,0.08,0.03,-0.13,-0.07,-0.07,0.15
nb_subjets,-0.01,1.00,0.04,-0.03,0.02,0.05,0.00,-0.01,-0.01,-0.01,0.01,0.13,-0.02,0.14,-0.04,-0.09,-0.03,0.05,-0.05,0.02,0.01,-0.03,-0.00,-0.00,0.01,0.01,-0.05,0.02,0.00,-0.03,-0.03,0.03,0.01
Family_size,0.02,0.04,1.00,-0.02,0.06,-0.02,0.05,-0.05,-0.02,-0.02,0.00,-0.02,-0.07,0.03,-0.00,-0.02,-0.05,-0.01,0.02,-0.08,0.07,0.05,0.04,0.11,-0.00,-0.05,0.03,-0.09,0.08,-0.00,-0.06,0.02,0.05
Internet,0.07,-0.03,-0.02,1.00,-0.01,0.05,0.08,0.11,0.20,0.16,0.26,-0.07,0.28,-0.06,0.01,0.04,-0.01,0.25,-0.25,0.00,-0.14,0.18,0.11,0.03,-0.06,0.22,0.04,0.09,-0.09,-0.00,0.00,-0.09,0.09
Study,0.01,0.02,0.06,-0.01,1.00,-0.04,-0.13,0.13,0.15,0.18,0.13,-0.08,-0.06,-0.03,-0.06,0.07,-0.02,0.04,-0.04,0.01,0.02,-0.04,-0.05,0.09,-0.04,0.08,0.01,0.03,-0.03,0.01,0.03,-0.08,0.05
Sleep,-0.07,0.05,-0.02,0.05,-0.04,1.00,0.14,0.02,0.03,0.04,0.07,0.01,-0.06,-0.01,0.06,-0.03,-0.01,-0.10,0.10,-0.03,0.03,0.02,0.00,-0.06,0.05,0.07,-0.05,-0.02,0.02,0.00,0.00,-0.01,0.01
Media,0.01,0.00,0.05,0.08,-0.13,0.14,1.00,-0.15,-0.08,-0.10,-0.09,-0.07,0.07,0.02,0.03,-0.04,-0.00,-0.03,0.03,0.01,-0.07,0.06,0.01,-0.04,0.08,0.00,0.05,-0.02,0.01,0.01,-0.02,0.05,-0.03
Note_avant,-0.21,-0.01,-0.05,0.11,0.13,0.02,-0.15,1.00,-0.05,-0.05,0.01,-0.01,0.12,-0.18,0.06,0.10,-0.04,0.24,-0.24,-0.03,-0.02,0.08,0.00,0.01,-0.10,0.10,-0.12,0.14,-0.02,-0.13,0.07,0.12,-0.20
Online_mode,0.17,-0.01,-0.02,0.20,0.15,0.03,-0.08,-0.05,1.00,0.72,0.56,-0.14,-0.07,0.09,-0.07,-0.03,-0.01,-0.11,0.11,-0.03,0.01,0.04,0.03,0.11,0.02,0.07,0.11,-0.01,-0.07,0.09,0.03,-0.44,0.40
doubts_online,0.13,-0.01,-0.02,0.16,0.18,0.04,-0.10,-0.05,0.72,1.00,0.56,-0.12,-0.09,0.07,-0.05,-0.02,-0.01,-0.13,0.13,-0.04,0.02,0.03,-0.02,0.11,-0.02,0.06,0.11,-0.01,-0.07,0.09,0.04,-0.45,0.39


In [14]:
#remarque : les éléves qui arrivent à participer activement en ligne et qui arrive à se renseigner avec  

In [19]:
corr = df_1.corr()
np.io.formats.style.Styler(corr,precision=2).background_gradient(cmap='copper_r')

,Age,nb_subjets,Family_size,Internet,Study,Sleep,Media,Note_avant,Online_mode,doubts_online,performance_online,Gender_Male,Home_location_Urban,Education_Post Graduate,Education_School,Education_Under Graduate,Device_used_Desktop,Device_used_Laptop,Device_used_Mobile,Economic_status_Middle Class,Economic_status_Poor,Economic_status_Rich,Sport_Yes,Monitoring_Yes,Gaming_Yes,separate_room_Yes,group_study_yes,interest_Both,interest_Practical,interest_Theory,Satisfaction_online_Average,Satisfaction_online_Bad,Satisfaction_online_Good
Age,1.00,-0.01,0.02,0.07,0.01,-0.07,0.01,-0.21,0.17,0.13,0.11,-0.04,0.00,0.42,-0.15,-0.24,0.02,0.02,-0.03,0.03,-0.05,0.02,0.02,-0.04,-0.01,0.02,0.11,0.08,0.03,-0.13,-0.07,-0.07,0.15
nb_subjets,-0.01,1.00,0.04,-0.03,0.02,0.05,0.00,-0.01,-0.01,-0.01,0.01,0.13,-0.02,0.14,-0.04,-0.09,-0.03,0.05,-0.05,0.02,0.01,-0.03,-0.00,-0.00,0.01,0.01,-0.05,0.02,0.00,-0.03,-0.03,0.03,0.01
Family_size,0.02,0.04,1.00,-0.02,0.06,-0.02,0.05,-0.05,-0.02,-0.02,0.00,-0.02,-0.07,0.03,-0.00,-0.02,-0.05,-0.01,0.02,-0.08,0.07,0.05,0.04,0.11,-0.00,-0.05,0.03,-0.09,0.08,-0.00,-0.06,0.02,0.05
Internet,0.07,-0.03,-0.02,1.00,-0.01,0.05,0.08,0.11,0.20,0.16,0.26,-0.07,0.28,-0.06,0.01,0.04,-0.01,0.25,-0.25,0.00,-0.14,0.18,0.11,0.03,-0.06,0.22,0.04,0.09,-0.09,-0.00,0.00,-0.09,0.09
Study,0.01,0.02,0.06,-0.01,1.00,-0.04,-0.13,0.13,0.15,0.18,0.13,-0.08,-0.06,-0.03,-0.06,0.07,-0.02,0.04,-0.04,0.01,0.02,-0.04,-0.05,0.09,-0.04,0.08,0.01,0.03,-0.03,0.01,0.03,-0.08,0.05
Sleep,-0.07,0.05,-0.02,0.05,-0.04,1.00,0.14,0.02,0.03,0.04,0.07,0.01,-0.06,-0.01,0.06,-0.03,-0.01,-0.10,0.10,-0.03,0.03,0.02,0.00,-0.06,0.05,0.07,-0.05,-0.02,0.02,0.00,0.00,-0.01,0.01
Media,0.01,0.00,0.05,0.08,-0.13,0.14,1.00,-0.15,-0.08,-0.10,-0.09,-0.07,0.07,0.02,0.03,-0.04,-0.00,-0.03,0.03,0.01,-0.07,0.06,0.01,-0.04,0.08,0.00,0.05,-0.02,0.01,0.01,-0.02,0.05,-0.03
Note_avant,-0.21,-0.01,-0.05,0.11,0.13,0.02,-0.15,1.00,-0.05,-0.05,0.01,-0.01,0.12,-0.18,0.06,0.10,-0.04,0.24,-0.24,-0.03,-0.02,0.08,0.00,0.01,-0.10,0.10,-0.12,0.14,-0.02,-0.13,0.07,0.12,-0.20
Online_mode,0.17,-0.01,-0.02,0.20,0.15,0.03,-0.08,-0.05,1.00,0.72,0.56,-0.14,-0.07,0.09,-0.07,-0.03,-0.01,-0.11,0.11,-0.03,0.01,0.04,0.03,0.11,0.02,0.07,0.11,-0.01,-0.07,0.09,0.03,-0.44,0.40
doubts_online,0.13,-0.01,-0.02,0.16,0.18,0.04,-0.10,-0.05,0.72,1.00,0.56,-0.12,-0.09,0.07,-0.05,-0.02,-0.01,-0.13,0.13,-0.04,0.02,0.03,-0.02,0.11,-0.02,0.06,0.11,-0.01,-0.07,0.09,0.04,-0.45,0.39


In [17]:
y = df["performance_online"].copy()

for i in range (len(y)):
    if y[i] < 7: y[i]= 0
    else : y[i]=1

df_1.drop(["performance_online"],axis=1,inplace=True)

In [18]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1028    1
1029    1
1030    0
1031    0
1032    1
Name: performance_online, Length: 1033, dtype: int64

In [19]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033 entries, 0 to 1032
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           1033 non-null   float64
 1   nb_subjets                    1033 non-null   float64
 2   Family_size                   1033 non-null   float64
 3   Internet                      1033 non-null   float64
 4   Study                         1033 non-null   float64
 5   Sleep                         1033 non-null   float64
 6   Media                         1033 non-null   float64
 7   Note_avant                    1033 non-null   float64
 8   Online_mode                   1033 non-null   float64
 9   doubts_online                 1033 non-null   float64
 10  Gender_Male                   1033 non-null   uint8  
 11  Home_location_Urban           1033 non-null   uint8  
 12  Education_Post Graduate       1033 non-null   uint8  
 13  Edu

In [20]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [21]:
import math
from random import seed, shuffle
SEED = 1122334455
seed(SEED)
np1.random.seed(SEED)


def reapeated_random(n_iter,df_1) :
    X = df_1.to_numpy(copy=True)
    y1 = y.to_numpy(copy=True)   
    matrise_result=[] # dim 1 -> interation 
                     # dim 2 -> 0: training ; 1:testing ; 2:validation
                     # dim 3 -> c1,c2,c3,c4,c5
    C_i=[0.001,1,100,1000,1]
    C_range=np1.shape(C_i)[0]
    for i in range (n_iter) :
        print(i,end='')
        n_size=np1.shape(X)[0]
        perm = list(range(0,n_size))
        shuffle(perm)
        X_syn1 = X[perm]
        y_syn1 = y[perm]
        #Generate training, testing and validation:
        n_train=int(0.5*n_size)
        n = np1.shape(X_syn1)[0]
        X_syn_training=X_syn1[0:n_train,:]
        y_syn_training=y_syn1[0:n_train]

        X_syn_testing=X_syn1[n_train:(n_train+math.floor((n-n_train)/2)),:]
        y_syn_testing=y_syn1[n_train:(n_train+math.floor((n-n_train)/2))]

        X_syn_validation=X_syn1[(n_train+math.floor((n-n_train)/2)):n,:]
        y_syn_validation=y_syn1[(n_train+math.floor((n-n_train)/2)):n]

        #resolution
        Accuracy_training=np1.empty([C_range])
        Accuracy_testing=np1.empty([C_range])
        Accuracy_validation=np1.empty([C_range])
        for i in range(C_range):
    
            svm_clf = SVC(kernel="linear", C=C_i[i])
            svm_clf.fit(X_syn_training, y_syn_training)
  
            y_pred_training=svm_clf.predict(X_syn_training)
            y_pred_testing=svm_clf.predict(X_syn_testing)
            y_pred_validation=svm_clf.predict(X_syn_validation)

            Accuracy_training[i]=(metrics.accuracy_score(y_syn_training, y_pred_training)*100)
            Accuracy_testing[i]=(metrics.accuracy_score(y_syn_testing, y_pred_testing)*100)
            Accuracy_validation[i]=(metrics.accuracy_score(y_syn_validation, y_pred_validation)*100)   
        
        matrise_result += [[Accuracy_training,Accuracy_testing,Accuracy_validation]]
    return matrise_result


In [22]:
def final_accuracy(matrice):
    Final_accuracy=[]
    for i in range (10):
        max_t=max(matrice[i][1])
        i_loc=np1.where(matrice[i][1]>=max_t)
        if len(i_loc[0])== 1 : 
            i_loc=i_loc[0][0]
        else : 
            acc = []
            for x in i_loc[0] : 
                acc += [matrice[i][2][x]]
            #print(acc)
            max_t = max(acc)
            i_acc = np1.where(acc>=max_t)
            i_loc= i_loc[0][i_acc[0][0]]
        
        Final_accuracy += [matrice[i][2][i_loc]]
    #print(Final_accuracy)
    print("La précision moyenne est  %.2f" %np1.mean(Final_accuracy))
    print("La variance de précision est %.2f" %np1.var(Final_accuracy))

In [24]:
df_2 = df_1[["Study","Sleep","Media","Note_avant"]].copy()
matrice = np1.array(reapeated_random(10,df_2))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  55.98
La variance de précision est 6.83


In [26]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  68.34
La variance de précision est 4.59


In [27]:
df_1.drop([ "nb_subjets" , "Sleep", "Note_avant","Home_location_Urban","Education_Post Graduate","Education_School" \
           ,"Education_Under Graduate","Economic_status_Middle Class","Economic_status_Poor","Economic_status_Rich", \
           "Sport_Yes", "Gaming_Yes","interest_Both"],axis=1,inplace=True)

In [28]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  68.15
La variance de précision est 2.15


In [29]:
df_1.drop(["group_study_yes","separate_room_Yes","Monitoring_Yes","Gender_Male", "Family_size", "Media","Device_used_Laptop" \
           ,"Device_used_Mobile","interest_Practical","interest_Theory","Device_used_Desktop"],axis=1,inplace=True)

In [30]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  69.73
La variance de précision est 7.94


In [31]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033 entries, 0 to 1032
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          1033 non-null   float64
 1   Internet                     1033 non-null   float64
 2   Study                        1033 non-null   float64
 3   Online_mode                  1033 non-null   float64
 4   doubts_online                1033 non-null   float64
 5   Satisfaction_online_Average  1033 non-null   uint8  
 6   Satisfaction_online_Bad      1033 non-null   uint8  
 7   Satisfaction_online_Good     1033 non-null   uint8  
dtypes: float64(5), uint8(3)
memory usage: 43.5 KB


In [32]:
df_1.drop(["Age","Study","Satisfaction_online_Average"],axis=1,inplace=True)

In [33]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033 entries, 0 to 1032
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Internet                  1033 non-null   float64
 1   Online_mode               1033 non-null   float64
 2   doubts_online             1033 non-null   float64
 3   Satisfaction_online_Bad   1033 non-null   uint8  
 4   Satisfaction_online_Good  1033 non-null   uint8  
dtypes: float64(3), uint8(2)
memory usage: 26.4 KB


In [34]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  69.88
La variance de précision est 6.74


In [35]:
df_1.drop(["Online_mode","Satisfaction_online_Bad","Satisfaction_online_Good"],axis=1,inplace=True)

In [36]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  67.99
La variance de précision est 7.47
